In [4]:
"""

------------------------------ Pivot Table ------------------------------

The below code enables the creation of a certain form of
'pivot' table through the function PivotTable(). It returns
a pandas.DataFrame() object with rows being the unique values
of the 'key' column provided and the columns specified in
'structs', which are to be passed in as arguemnts to the
PivotTable() function.

Function:-

PivotTable(data, key, *structs)

data - expects a pandas.DataFrame object of the original data
key - expects a string name of the column you want to insepct
structs - expects a one or multiple list objects containing two
          things: the string name of the column, and the operation
          you want to see being applied on that column. The
          options for different operations is given below.

NOTE: Only the 'mode' operation should be used if the struct column
      has non-numerical values.

Operations :-

'avg': provides the average of the column (excludes any NaN values)
'sum': provides the total sum of the column (excludes any NaN values)
'Count': provides the total number of data points.
'binCount': simply subtracts the sum from the len, useful if the
            data is binary
'std': provides the standard deviation of the column
'mode': provides the mode of the column
'median': provides the median of the column
'rmode': provides the least occuring item in the column
'max': provides the maximum valued item in the column
'min': provides the minimum valued item in the column

Example usage :-

Df = PivotTable(data, 'loan_status', ['grades', 'mode'], ['annual_inc', 'avg'])
print(Df)

----------------------------------------------------------------------------
"""


import re
import numpy
import pandas
import warnings
from collections import Counter
from scipy.stats import skew
from scipy.stats import pointbiserialr
import matplotlib.pyplot as plt

warnings.simplefilter('ignore')

In [45]:
def are_all_nums_or_floats(lst):
    return all(isinstance(x, (int, float)) for x in lst)

def is_duplicate (col) :
  col = col.to_numpy()
  return (col[0] == col).all()

def get_unique (frame, col) :
  cols = frame[col].unique()
  print(f'Length: {len(cols)}')
  for i in cols :
    print(i)

def drop_column (dropped, data, col) :
  if type(col) == type(list()):
    for i in col :
      dropped[i] = data.pop(i)
    return
  else:
    dropped[col] = data.pop(col)

def clean_year(year):
  if type(year) != type("") :
    return None
  if "10+" in year :
    return float(re.sub(r'(\d+)\s*\+\s*years', r'\1', year))
  if "<" in year :
    return float(re.sub(r'<\s*1\s*years*', '0.5', year))
  else :
    return float(re.sub(r'\s*years*$', '', year))

def Operations (nlist, operation=None) :
  if operation == 'avg' and are_all_nums_or_floats(nlist):
    return numpy.nanmean(numpy.array(nlist))
  if operation == 'sum' and are_all_nums_or_floats(nlist):
    return numpy.nansum(numpy.array(nlist))
  if operation == 'Count':
    return len(nlist)
  if operation == 'binCount' and are_all_nums_or_floats(nlist):
    return len(nlist) - sum(nlist)
  if operation == 'mode':
    counts = Counter(nlist)
    return max(counts, key=counts.get)
  if operation == 'rmode':
    counts = Counter(nlist)
    return min(counts, key=counts.get)
  if operation == 'median' and are_all_nums_or_floats(nlist):
    return numpy.median(numpy.array(nlist))
  if operation == 'std' and are_all_nums_or_floats(nlist):
    return numpy.std(numpy.array(nlist))
  if operation == 'max' and are_all_nums_or_floats(nlist):
    return numpy.max(numpy.array(nlist))
  if operation == 'min' and are_all_nums_or_floats(nlist):
    return numpy.min(numpy.array(nlist))
  if operation is None:
    return nlist

class Map :

  def __init__(self, data, key) :
    self.data = data
    self.keys = list(data[key])

  def operationCol (self, value, operations) :
    self.map, self.freq = dict(), dict()
    values = list(self.data[value])
    for i in range(len(self.keys)) :
      if self.keys[i] not in self.map.keys () :
        self.freq[self.keys[i]] = 1
        self.map[self.keys[i]] = [values[i]]
      else :
        self.freq[self.keys[i]] += 1
        self.map[self.keys[i]].append(values[i])
    for i in self.map.keys() :
      self.map[i] = Operations(self.map[i], operation=operations)
    return list(self.map.values())

def PivotTable (data, key, *structs) :
  if structs == [] or key == '':
    print('Empty List!', end=" ")
    return
  else :
    map = Map (data, key)
    df = pandas.DataFrame()
    for col in structs :
      df[f"{col[0]}({col[1]})"] = map.operationCol(col[0], col[1])
    df.index = map.map.keys()
    df.insert(0, 'Freq', map.freq.values())
    return df

"""
------------------------------- Default Rate -------------------------------

The default rate is a derived metric, essentially calculated by taking the
number of people who have defaulted in that particular row, and dividing it
by the total number of people who've taken the loan in that category.

----------------------------------------------------------------------------
"""

def DefaultRate (data, key) :
  df = PivotTable (data, key, ['loan_status', 'binCount'], ['annual_inc', 'median'])
  df['DefaultRate(%)'] = (df['loan_status(binCount)'] / df['Freq']) * 100
  print(df)
  if key == 'income_bracket' or key == 'grade' or key == 'sub_grade':
    print(f"Skewness: {skew(df['DefaultRate(%)'], bias=False)}")
  print()
  fig = plt.figure(figsize=(10, 6), facecolor='#364449', edgecolor='white')
  ax = fig.add_subplot(1, 1, 1)
  ax.set_facecolor('#364449')
  bars = plt.bar(df.index, df['DefaultRate(%)'], color='skyblue')
  for bar in bars:
    bar.set_color('white')  # Bar color
  plt.xticks(color='white')  # X-axis tick color
  plt.yticks(color='white')  # Y-axis tick color
  plt.xlabel(key, color='white')  # X-axis label color
  plt.ylabel('Default Rate in %', color='white')  # Y-axis label color
  plt.title('Defaults', color='white')  # Plot title color

  plt.show()

In [14]:
data = pandas.read_csv('loan.csv')
columns = list(data.columns)
dropped_columns = pandas.DataFrame()

"""
----------------------------- Data Engineering -----------------------------

The following modifications where made :
 > Removal of columns with over 25000 NULL values or with too many duplicate
   values.
 > Removal of columns with too many unique values, where proper analysis is not
   possible, there being too much noise.
 > Removing people who have neither paid nor defaulted ('Current' in their status).
 > Changing the data type of 'emp_length' to numerical years, where '<1 years'
   is taken to be as 0.5, while '10+ years' is simply 10.
 > The column, 'term', is also converted to binary: 0 for 36 months and 1 for 60.
 > 'loan_status' is also converted to binary: 0 for 'Charged Off' and 1 for
   'Fully paid'.
 > 'verification_status' is also converted to binary: 0 for 'Not Verified' and 1
   otherwise.
 > The 'annual_inc' column was used to create a new column of income brackets,
   where incomes where grouped together like so:
   (['0-60000', '60000-120000', '120000-240000', '>240000'])

----------------------------------------------------------------------------
"""

## Removing columns with too many NULL values
for i in range(len(columns)) :
  if data[columns[i]].isna().sum() > 25000 or is_duplicate(data[columns[i]]):
    drop_column(dropped_columns, data, columns[i])

## Removing unwanted columns
drop_column(dropped_columns, data, ['id', 'member_id', 'emp_title', 'url', 'desc', 'title', 'zip_code'])

## Splitting up annual income into income-brackets
bins = [0, 60000, 120000, 240000, float('inf')]
labels = ['0-60000', '60000-120000', '120000-240000', '>240000']

## Data engineering
data = data[data['loan_status'] != 'Current']
data['int_rate'] = [float(i.replace("%","")) for i in data['int_rate']]
data['emp_length'] = [clean_year(i) for i in data['emp_length']]
data['loan_status'] = [0 if i == 'Charged Off' else 1 for i in data['loan_status']]
data['term'] = [0 if i == ' 36 months' else 1 for i in data['term']]
data['verification_status'] = [0 if i == 'Not Verified' else 1 for i in data['verification_status']]
data['income_bracket'] = pandas.cut(data['annual_inc'], bins=bins, labels=labels, right=False)

In [ ]:
keyVariable = 'loan_status'
valueVariables = [
    ['funded_amnt', 'sum'],
    ['funded_amnt_inv', 'sum'],
    ['term', 'binCount'],
    ['int_rate', 'avg'],
    ['installment', 'avg'],
    ['grade', 'mode'],
    ['sub_grade', 'mode'],
    ['emp_length', 'avg'],
    ['home_ownership', 'mode'],
    ['annual_inc', 'median'],
    ['purpose', 'mode'],
    ['pub_rec_bankruptcies', 'sum'],
    ['pub_rec', 'sum'],
    ['dti', 'avg'],
    ['open_acc', 'max'],
    ['addr_state', 'mode']
]

Df = PivotTable (data, keyVariable, *valueVariables)
Df.insert(3, 'investors(%)', Df['funded_amnt_inv(sum)']/Df['funded_amnt(sum)'])
Df['Bankruptcy(%)'] = Df['pub_rec_bankruptcies(sum)'] / Df['Freq'] * 100
Df['Derogatory_Records(%)'] = Df['pub_rec(sum)'] / Df['Freq'] * 100
print(Df)

In [ ]:
df = PivotTable(data, 'loan_status', ['total_pymnt', 'avg'], ['funded_amnt', 'avg'])
df['remaining_amnt'] = (df['funded_amnt(avg)'] - df['total_pymnt(avg)']).apply(lambda x: max(0, x))
df['remaining(%)'] = (df['remaining_amnt'] / df['funded_amnt(avg)']) * 100
print(df)

In [ ]:
DefaultRate(data, 'home_ownership')

In [ ]:
DefaultRate(data, 'income_bracket')

In [ ]:
DefaultRate(data, 'purpose')

In [ ]:
DefaultRate(data, 'addr_state')

In [ ]:
DefaultRate(data, 'grade')

In [ ]:
DefaultRate(data, 'sub_grade')

In [ ]:
newData = data[data['pub_rec_bankruptcies'].notna()]
df = PivotTable(newData, 'pub_rec_bankruptcies', ['int_rate', 'avg'])
print(df)